In [4]:
import os
from sqlalchemy import create_engine
import pandas as pd

# Set credentials 
os.environ['PG_HOST'] = 'isba-dev-02.cmb4w8cmqb26.us-east-1.rds.amazonaws.com'
os.environ['PG_USER'] = 'postgres'
os.environ['PG_PASSWORD'] = 'isba_4715'
os.environ['PG_DB'] = 'data_engineer_project'

host = os.getenv('PG_HOST')
user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')
database = os.getenv('PG_DB')

connection_string = f"postgresql+psycopg2://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)
pd.set_option('display.max_rows', None)

test_query = '''
SELECT COUNT(*) FROM raw."Active_Players"
limit 10;
'''
pd.read_sql(test_query, engine)

,count
0,65


# Descriptive Business Question: Does League of Legends have more players now compared to 2020?

In [5]:
sql_query = """
WITH filtered AS (
    SELECT 
        "Month",
        "Avg Monthly Players",
        "Peak Players",
        CASE 
            WHEN "Month" IN ('20-Jan', '20-Feb', '20-Mar', '20-Apr', '20-May', '20-Jun',
                             '20-Jul', '20-Aug', '20-Sep', '20-Oct', '20-Nov', '20-Dec')
                THEN '2020'
            WHEN "Month" IN ('24-Jan', '24-Feb', '24-Mar', '24-Apr', '24-May', '24-Jun',
                             '24-Jul', '24-Aug', '24-Sep', '24-Oct', '24-Nov', '24-Dec',
                             '25-Jan', '25-Feb', '25-Mar', 'Last 30 Days')
                THEN 'Now'
            ELSE 'Other'
        END AS period
    FROM raw."Active_Players"
)
SELECT 
    period,
    ROUND(AVG("Avg Monthly Players")::numeric) AS avg_players,
    ROUND(AVG("Peak Players")::numeric) AS avg_peak_players
FROM filtered
WHERE period IN ('2020', 'Now')
GROUP BY period
ORDER BY period;
"""


# Execute
query_result = pd.read_sql(sql_query, engine)
query_result

,period,avg_players,avg_peak_players
0,2020,48900955.0,9943194.0
1,Now,47592416.0,10063837.0


# Insight
1. League of Legends has a slightly smaller player base, ↓ ~1.3 million fewer players per month now compared to 2020
2. However, peak players counts are slightly higher, ↑ ~120,000 more peak players

# Recommendation
1. Investigate what drives peak activity in recent months, since 2025 is only about half way through. The peak
might have to do with events or updates, seasonal spikes and so on. 
2. Think of ways to extend engagement beyond peaks, like retention events. 

# Prediction
1. If the pattern continues, the player base may be shifting from a steady engagement to spike-driven activity. 

# Diagnostic Business Question: Has player engagement become more burst-like over time? (We define this by tracking the ratio of average monthly players to peak players.)

In [7]:
sql_query = """
WITH ranked_months AS (
    SELECT 
        "Month",
        "Avg Monthly Players",
        "Peak Players",
        ROUND("Avg Monthly Players"::numeric / NULLIF("Peak Players", 0), 2) AS avg_to_peak_ratio,
        ROW_NUMBER() OVER (ORDER BY "Month") AS rn_asc,
        ROW_NUMBER() OVER (ORDER BY "Month" DESC) AS rn_desc
    FROM raw."Active_Players"
)
SELECT "Month", avg_to_peak_ratio
FROM ranked_months
WHERE rn_asc = 1 OR rn_desc = 1
ORDER BY "Month";
"""

# Execute
query_result = pd.read_sql(sql_query, engine)
query_result

,Month,avg_to_peak_ratio
0,20-Apr,4.92
1,Last 30 Days,3.70


# Insight
1. average to peak ratio dropped from 4.92 in April 2020 to 3.70 in the last 30 days
2. Players are now showing up in shorter, high-concentration sessions rather than playing consistently across the month

# Recommendation 
 Support the peak-time events, and also think of ways to raie baseline engagement. 

# Prediction 
If this trend continues, future engagement will be even more event-driven, 
and success will depend more on the quality and timing of content drops but not long-term play cycles.